In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
from scipy.stats import norm, multivariate_normal
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy
import Tasmanian
import matplotlib.pyplot as plt

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

CPU times: total: 3.38 s
Wall time: 3.85 s


In [2]:
def sg_integrate(func, a, b, dim, depth = 20, seed = 1):
    np.random.seed(seed)

    #Get grid points and weights
    grid_p = Tasmanian.SparseGrid()
    grid_p.makeGlobalGrid(dim, 0, depth, "hyperbolic", 'gauss-jacobi')
    points_p = grid_p.getPoints()
    # print(len(points_p))
    weights_p = grid_p.getQuadratureWeights()

    if dim == 2:
        y = func(points_p[:,0], points_p[:,1])
    elif dim ==5:
        y = func(points_p[:,0], points_p[:,1], points_p[:,2],points_p[:,3],points_p[:,4])

    integ = y@weights_p.T

    return integ

def mc_integrate(func, a, b, dim, n, seed = 1):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    #Initialize total ei  
    # n = math.ceil((norm.ppf(1-0.05/2)/0.05)**2)  
    np.random.seed(seed)

    #Get random variable
    x_list = np.random.uniform(a, b, (n, dim))

    if dim == 2:
        y = func(x_list[:,0], x_list[:,1])
    elif dim ==5:
        y = func(x_list[:,0], x_list[:,1], x_list[:,2],x_list[:,3],x_list[:,4])

    y_mean =  y.sum()/len(y)
    domain = np.power(b-a, dim)

    integ = domain * y_mean

    # Perform bootstrapping
    bootstrap_vars = bootstrap(y, ns=1000, alpha=0.05, seed=seed)

    return integ, np.array(bootstrap_vars)*domain

def bootstrap(pilot_sample, ns=100, alpha=0.05, seed = 1):
    # pilot_sample has one column per rv, one row per observation
    # alpha is the level of significance; 0.05 for 95% confidence interval
    quantiles = np.array([alpha*0.5, 1.0-alpha*0.5])

    #Set seed
    if seed is not None:
        np.random.seed(seed)

    #Determine mean of all original samples and its shape
    theta_orig = np.mean(pilot_sample,axis=0)

    #Initialize bootstrap samples as zeros
    theta_bs = np.zeros(tuple([ns]+list(theta_orig.shape)))

    #Create bootstrap samples
    for ibs in range(ns):
        samples = np.random.choice(pilot_sample, size= pilot_sample.shape[0], replace=True)
        theta_bs[ibs,...] = np.mean(samples, axis = 0)

    # percentile CI
    CI_percentile = np.quantile(theta_bs, quantiles, 0)

    # return theta_orig, theta_bs, CI_percentile
    return CI_percentile


#Create Test 2D Functions
def func1(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    return 1 - x1**2 - x2**2

def func2(x1, x2):
    return np.maximum(func1(x1,x2), 0)

def func3(x1, x2):
    return 0.5*(np.sqrt(func1(x1, x2)**2 + 1e-5) + func1(x1, x2))

def func4(x1, x2):
    return np.sin(x1)*x2

def func5(x1, x2):
    return np.maximum(func4(x1,x2), 0)

def func6(x1, x2):
    return 0.5*(np.sqrt(func4(x1, x2)**2 + 1e-5) + func4(x1, x2))

def func7(x1, x2, x3, x4, x5):
    return 1 - x1**2 - x2**2 - x3**2 - x4**2 - x5**2

def func8(x1, x2, x3, x4, x5):
    return np.maximum(func7(x1, x2, x3, x4, x5), 0)

def func9(x1, x2, x3, x4, x5):
    return 0.5*(np.sqrt(func7(x1, x2, x3, x4, x5)**2 + 1e-5) + func7(x1, x2, x3, x4, x5))

In [3]:
column_names = ["scipy", "mc n=100", "mc n=1000", "mc n=10000", "sg depth=5", "sg depth=10", "sg depth=20"]
results_df = pd.DataFrame(columns=column_names)
# func_list = [func1, func2, func3, func4, func5, func6]
func_list = [func1, func2, func3, func4, func5, func6, func7, func8, func9]

l_bound = -1
u_bound = 1
seed = 1
depth1 = 5
depth2 = 10
depth3 = 20
n1 = 100
n2 = 1000
n3 = 10000

for i in range(len(func_list)):
    func = func_list[i]

    if i > 5:
        bounds = np.full((5, 2), [l_bound, u_bound])
        if i > 6:
            z_scipy = "n/a"
        else:
            z_scipy = integrate.nquad(func, bounds)[0]
        dim =5
    else:
        z_scipy = integrate.dblquad(func, l_bound, u_bound, l_bound, u_bound)[0]
        dim =2

    z_mc1, z_mc_ci1= mc_integrate(func, l_bound, u_bound, dim, n1, seed = seed)
    z_mc2, z_mc_ci2= mc_integrate(func, l_bound, u_bound, dim, n2, seed = seed)
    z_mc3, z_mc_ci3= mc_integrate(func, l_bound, u_bound, dim, n3, seed = seed)
    z_sg1 = sg_integrate(func, l_bound, u_bound, dim, depth1, seed = seed)
    z_sg2 = sg_integrate(func, l_bound, u_bound, dim, depth2, seed = seed)
    z_sg3 = sg_integrate(func, l_bound, u_bound, dim, depth3, seed = seed)

    iter_df = pd.DataFrame(columns=column_names)
    integ_results = [z_scipy, z_mc1, z_mc2, z_mc3, z_sg1, z_sg2, z_sg3]
    # Add the new row to the DataFrame
    iter_df.loc[0] = integ_results
    results_df = pd.concat([results_df.astype(iter_df.dtypes), iter_df], ignore_index=True)

print(results_df)


c:\Users\mcarlozo\AppData\Local\anaconda3\envs\Toy_Problem_env\Lib\site-packages\scipy\integrate\_quadpack_py.py:1233: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


       scipy   mc n=100  mc n=1000  mc n=10000  sg depth=5  sg depth=10  \
0   1.333333   1.006297   1.288451    1.336877    1.333333     1.333333   
1   1.570795   1.288276   1.533663    1.569565    1.333333     1.873080   
2   1.570888   1.288369   1.533761    1.569660    1.333379     1.873007   
3        0.0   0.137445  -0.002983    0.000148    0.000000     0.000000   
4   0.459698   0.576202   0.466506    0.458559    0.630242     0.430468   
5   0.459953   0.576323   0.466752    0.458818    0.630274     0.432838   
6 -21.333333 -24.319122 -21.044289  -21.543264  -21.333333   -21.333333   
7        n/a   1.231010   1.553981    1.512449  -21.333333    21.846363   
8        n/a   1.231592   1.554520    1.512910  -21.331940    22.339082   

   sg depth=20  
0     1.333333  
1     1.456093  
2     1.456331  
3     0.000000  
4     0.450998  
5     0.452638  
6   -21.333333  
7  -103.181580  
8  -103.607586  


In [4]:
# results_df.to_csv("/Users/mcarlozo/Documents/Graduate_Research/Toy_Problem/mc_test_all.csv")

In [23]:
def sg_integrate_gauss(func, dim, depth = 20, seed = 1):
    np.random.seed(seed)

    #Get grid points and weights
    grid_p = Tasmanian.SparseGrid()
    grid_p.makeGlobalGrid(dim, 0, depth, "hyperbolic", 'gauss-hermite')
    points_p = grid_p.getPoints()
    points_p = np.sqrt(2)*points_p
    weights_p = grid_p.getQuadratureWeights()

    if dim == 2:
        y = func(points_p[:,0], points_p[:,1])
    elif dim ==5:
        y = func(points_p[:,0], points_p[:,1], points_p[:,2],points_p[:,3],points_p[:,4])

    integ = (1/np.pi)*y@weights_p.T

    return integ

def mc_integrate_gauss(func, dim, n, seed = 1):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    #Initialize total ei  
    # n = math.ceil((norm.ppf(1-0.05/2)/0.05)**2)  
    np.random.seed(seed)

    #Get random variable
    x_list = np.random.multivariate_normal(np.zeros(dim), np.eye(dim), n)

    if dim == 2:
        y = func(x_list[:,0], x_list[:,1])
    elif dim ==5:
        y = func(x_list[:,0], x_list[:,1], x_list[:,2],x_list[:,3],x_list[:,4])

    y_mean =  np.mean(y)

    integ = y_mean

    # Perform bootstrapping
    bootstrap_vars = bootstrap(y, ns=1000, alpha=0.05, seed=seed)

    return integ, np.array(bootstrap_vars)

def func1_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func1(x1,x2)*pdf

def func2_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func2(x1,x2)*pdf

def func3_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func3(x1,x2)*pdf

def func4_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func4(x1,x2)*pdf

def func5_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func5(x1,x2)*pdf

def func6_pdf(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func6(x1,x2)*pdf

def func7_pdf(x1, x2, x3, x4, x5):
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func7(x1,x2,x3,x4,x5)*pdf

def func8_pdf(x1, x2, x3, x4, x5):
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func8(x1,x2,x3,x4,x5)*pdf

def func9_pdf(x1, x2, x3, x4, x5):
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
    pdf = multivariate_normal.pdf(vars, mean=np.zeros(vars.shape[1]), cov=np.eye(vars.shape[1]))
    return func9(x1,x2,x3,x4,x5)*pdf

In [28]:
column_names = ["scipy", "mc n=100", "mc n=1000", "mc n=10000", "sg depth=5", "sg depth=10", "sg depth=20"]
results_df = pd.DataFrame(columns=column_names)
func_list = [func1, func2, func3, func4, func5, func6, func7, func8, func9]
func_list2 = [func1_pdf, func2_pdf, func3_pdf, func4_pdf, func5_pdf, func6_pdf, func7_pdf, func8_pdf, func9_pdf] # func7_pdf, func8_pdf, func9_pdf
# func_list = [func1, func2, func3, func4, func5, func6, func7, func8, func9]

l_bound = -1
u_bound = 1
seed = 1
depth1 = 5
depth2 = 10
depth3 = 20
n1 = 100
n2 = 1000
n3 = 10000

for i in range(len(func_list)):
    func = func_list[i]
    func_pdf = func_list2[i]

    if i > 5:
        bounds = np.full((5, 2), [-np.inf, np.inf])
        if i > 6:
            z_scipy = "n/a"
        else:
            z_scipy = integrate.nquad(func_pdf, bounds)[0]
        dim =5
    else:
        z_scipy = integrate.dblquad(func_pdf, -np.inf, np.inf, -np.inf, np.inf)[0]
        dim =2

    z_mc1, z_mc_ci1= mc_integrate_gauss(func, dim, n1, seed = seed)
    z_mc2, z_mc_ci2= mc_integrate_gauss(func, dim, n2, seed = seed)
    z_mc3, z_mc_ci3= mc_integrate_gauss(func, dim, n3, seed = seed)
    z_sg1 = sg_integrate_gauss(func, dim, depth1, seed = seed)
    z_sg2 = sg_integrate_gauss(func, dim, depth2, seed = seed)
    z_sg3 = sg_integrate_gauss(func, dim, depth3, seed = seed)

    iter_df = pd.DataFrame(columns=column_names)
    integ_results = [z_scipy, z_mc1, z_mc2, z_mc3, z_sg1, z_sg2, z_sg3]
    # Add the new row to the DataFrame
    iter_df.loc[0] = integ_results
    results_df = pd.concat([results_df.astype(iter_df.dtypes), iter_df], ignore_index=True)

print(results_df)

c:\Users\mcarlozo\AppData\Local\anaconda3\envs\Toy_Problem_env\Lib\site-packages\scipy\integrate\_quadpack_py.py:1233: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


In [ ]:
results_df.to_csv("/Users/mcarlozo/Documents/Graduate_Research/Toy_Problem/mc_test_gauss_all2.csv")

In [8]:
cs_name_val = 1
meth_name_enum = 5
CS_name  = CS_name_enum(cs_name_val)
param_name_str = "t1t2" #set_param_str(cs_name_val)
indecies_to_consider = set_idcs_to_consider(cs_name_val, param_name_str)
meth_name = Method_name_enum(meth_name_enum)
method = GPBO_Methods(meth_name)

ep0 = 1
ep_enum = Ep_enum(1)
sep_fact = 1.0
normalize = False
noise_mean = 0
noise_std = 0.01
# noise_std = 0.0
kernel = Kernel_enum(1)
lenscl = None
outputscl = 1 #outpulscl tuning is critical for log scaled obj fxns not to terminate early w/ regret (stdv affected)
retrain_GP = 1
reoptimize_obj = 1
bo_iter_tot = 1
bo_run_tot = 1
save_data = False
seed = 5
ei_tol = 1e-6
obj_tol = 1e-6
DateTime = None

num_x_data = 5
gen_meth_x = Gen_meth_enum(2) #Note: Has to be the same for validation and sim data
num_theta_data = 10*len(indecies_to_consider)
num_theta_data_val = 200
gen_meth_theta = Gen_meth_enum(1)
gen_meth_theta_val = Gen_meth_enum(1)
gen_heat_map_data = False

In [9]:
#Get training data
simulator = simulator_helper_test_fxns(CS_name, indecies_to_consider, noise_mean, noise_std, seed)

#Calculate minimum Muller potential
min_Mul = solve_pyomo_Muller_min(param_name_str, verbose = False)

#Generate Exp Data
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)

#Generate Sim Data
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, False)

#Generate sse_sim_data from new sim and exp_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate validation data
val_data = simulator.gen_sim_data(num_theta_data_val, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, True)
val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)

#Set Cs_params and Simulator
cs_name = CS_name.name + "_BO_method_" + meth_name.name + "_sep_fact_" + str(round(sep_fact,2))
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl, retrain_GP, 
                                reoptimize_obj, gen_heat_map_data, bo_iter_tot, bo_run_tot, save_data, DateTime, 
                                seed, ei_tol, obj_tol)

#Initialize Driver
ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, 
                     ep_bias, gen_meth_theta)

In [10]:
#Make emulator
if driver.method.emulator == False:
    all_gp_data = driver.sim_sse_data
    all_val_data = driver.val_sse_data
    gp_emulator = Type_1_GP_Emulator(all_gp_data, all_val_data, None, None, None, driver.cs_params.kernel, 
                                     driver.cs_params.lenscl, driver.simulator.noise_std, driver.cs_params.outputscl, 
                                     driver.cs_params.retrain_GP, driver.cs_params.seed, None, None, None, None)
else:
    all_gp_data = driver.sim_data
    all_val_data = driver.val_data
    gp_emulator = Type_2_GP_Emulator(all_gp_data, all_val_data, None, None, None, driver.cs_params.kernel, 
                                     driver.cs_params.lenscl, driver.simulator.noise_std, driver.cs_params.outputscl, 
                                     driver.cs_params.retrain_GP, driver.cs_params.seed, None, None, None, None)
    
driver.gp_emulator = gp_emulator
#Set train_test data
train_data, test_data = driver.gp_emulator.set_train_test_data(driver.cs_params.sep_fact, driver.cs_params.seed)
        
#Initilize gp model
gp_model = driver.gp_emulator.set_gp_model()
driver.gp_emulator.train_gp(gp_model)

In [11]:
def ei_func(random_var, best_error, y_target, gp_mean, gp_var):
    #Create a mask for values where pred_stdev >= 0 (Here approximation includes domain stdev >= 0) 
    pos_stdev_mask = (gp_var >= 0)

    #Assuming all standard deviations are not zero
    if np.any(pos_stdev_mask):
        #Get indices and values where stdev > 0
        valid_indices = np.where(pos_stdev_mask)[0]
        gp_stdev_val = np.sqrt(gp_var[valid_indices])
        gp_mean_val = gp_mean[valid_indices]
        y_target_val = y_target[valid_indices]
        mean_min_y = y_target_val - gp_mean_val
    
        # Calculate gp_var multiplied by points_p
        gp_stdev_rand_var = gp_stdev_val * random_var
        gp_stdev_rand_var = gp_stdev_val * random_var

        # Calculate the SSE for all data points simultaneously
        sse_temp = np.sum((mean_min_y[:, np.newaxis].T - gp_stdev_rand_var)**2, axis=1)

        # Apply max operator (equivalent to max[(best_error*ep) - SSE_Temp,0])
        improvement = np.maximum(best_error - sse_temp, 0).reshape(-1,1)

        # Calculate EI_temp using vectorized operations
        ei_temp = improvement.flatten()
        
        # Calculate the multivariate normal pdf for each row in 'epsilon'
        # mvn = multivariate_normal.pdf(random_var, mean=np.zeros(random_var.shape[1]), cov=np.eye(random_var.shape[1]))

        # ei_temp = ei_temp*mvn

    else:
        ei_temp = 0
        
    return ei_temp

In [12]:
#Code Courtesy of Ryan Smith
def bootstrap(pilot_sample, ns=100, alpha=0.05, seed = 1):
    # pilot_sample has one column per rv, one row per observation
    # alpha is the level of significance; 0.05 for 95% confidence interval
    quantiles = np.array([alpha*0.5, 1.0-alpha*0.5])

    #Set seed
    if seed is not None:
        np.random.seed(seed)

    #Determine mean of all original samples and its shape
    theta_orig = np.mean(pilot_sample,axis=0)

    #Initialize bootstrap samples as zeros
    theta_bs = np.zeros(tuple([ns]+list(theta_orig.shape)))

    #Create bootstrap samples
    for ibs in range(ns):
        samples = np.random.choice(pilot_sample, size= pilot_sample.shape[0], replace=True)
        theta_bs[ibs,...] = np.mean(samples, axis = 0)

    # percentile CI
    CI_percentile = np.quantile(theta_bs, quantiles, 0)

    # return theta_orig, theta_bs, CI_percentile
    return CI_percentile

In [13]:
#Test MC integration
def mc_integrate(func, driver, dim):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    #Initialize total ei
    n = math.ceil((norm.ppf(1-0.05/2)/0.05)**2)
    
    np.random.seed(seed)
    
    integ_theta = np.zeros(len(driver.gp_emulator.gp_val_data.get_unique_theta()))
    bs_vars = []
    for i in range(len(driver.gp_emulator.gp_val_data.get_unique_theta())):
        #Calcuate best error
        if driver.method.emulator == False:
            #Type 1 best error is inferred from training data 
            best_error, be_theta = driver.gp_emulator.calc_best_error()
            best_errors_x = None
        else:
            #Type 2 best error must be calculated given the experimental data
            best_error, be_theta, best_errors_x = driver.gp_emulator.calc_best_error(driver.method, driver.exp_data)
        #Evaluate GP for validation data
        y_sim = driver.exp_data.y_vals
        gp_mean_all, gp_var_all = driver.gp_emulator.eval_gp_mean_var_val()
        gp_mean = gp_mean_all[i*len(y_sim):i*len(y_sim)+len(y_sim)]
        gp_var = gp_var_all[i*len(y_sim):i*len(y_sim)+len(y_sim)]
        #Get random variable
        random_var = np.random.multivariate_normal(np.zeros(dim), np.eye(dim), n)
        #Calc EI
        ei = func(random_var, best_error, y_sim, gp_mean, gp_var)
        ei_mean = np.average(ei) #y.sum()/len(y)
        domain = 1
        
        #Calc monte carlo integrand for each theta and add it to the total
        integ = domain * ei_mean
        integ_theta[i] = integ

        # Perform bootstrapping
        bootstrap_vars = bootstrap(ei, ns=100, alpha=0.05, seed=seed)
        bs_vars.append(bootstrap_vars)

    return integ_theta, np.array(bs_vars)


# n = 115813
#Fill in f_args with data from a run
ei_mc, ci = mc_integrate(ei_func, driver, len(driver.exp_data.y_vals))
print(ei_mc[ei_mc > 0])
print(ci[(ci > 0).all(axis=1)])

KeyboardInterrupt: 

In [ ]:
#Test Sparse Grid Integration
#Calcuate best error
if driver.method.emulator == False:
    #Type 1 best error is inferred from training data 
    best_error_metrics = driver.gp_emulator.calc_best_error()
    best_errors_x = None
else:
    #Type 2 best error must be calculated given the experimental data
    best_error_metrics = driver.gp_emulator.calc_best_error(driver.method, driver.exp_data)
#Set be in ep bias class
driver.ep_bias.best_error = best_error_metrics[0]
driver.ep_bias.set_ep()
#Calculate EI for validation data
if driver.method.emulator == False:
    ei_output = driver.gp_emulator.eval_ei_val(driver.exp_data, driver.ep_bias, best_error_metrics)
else:
    ei_output = driver.gp_emulator.eval_ei_val(driver.exp_data, driver.ep_bias, best_error_metrics, driver.method)

ei_sparse = ei_output[0]
print(ei_sparse)

In [ ]:
heat_map_dict = driver.create_heat_map_param_data(n_points_set = 20)
heat_map = heat_map_dict[list(heat_map_dict.keys())[0]]

#Init EI array list
#Loop over n (100, 1000, and 10000)
    #Init EI array
    #Loop over each theta in heat map data
        #Get EI From mc and save to ei array 
    #Save EI array to EI array list
#Loop over depth for sparse grid (5, 10, 20)
    #Init EI array
    #Loop over each theta in heat map data
        #Get EI from Sparse Grid (Depth = 5, 10, 20)
    #Save EI array to EI array list
#Return ei of each theta for each method